### Recognizing hand-written digits with a single layer softmax classifier 

This is our second experiment with the <a href="http://yann.lecun.com/exdb/mnist/">MNIST </a> dataset for recognizing handwritten digits using neural networks. There are 50,000 training examples in the MNIST dataset. Each digit is represented by a 28 by 28 pixel grayscale image. For input representation, we need to flatten this matrix into a vector representation of size 28 * 28 = 784. Here we consider the simplest neural networks architecture possible, which is a single layer softmax classifier, meaning there is only a single output layer in the network. The output layer (alternatively called the softmax layer here) in this network consists of 10 neurons, each representing a digit in the range 0 to 9.

Neurons in the output layer act in the same way. Each does a weighted sum of all their inputs, adds a constant which we call <i>bias</i> and then feeds the result into a non-linear activation function. For classification, the softmax function with L = 1 is usually preferred although several possible alternatives will be mentioned later. $L_n$ denotes the weighted sum of all pixels + bias in each neuron:

$$softmax(L_n) = \frac{e^{L_n}}{\left \| e^L \right \|}
$$

In other words, the softmax function takes an N-dimensional vector of real values (N being the number of neurons in the layer, in this case N = 10) and maps them to another N-dimensional vector with values in the range (0, 1) that add up to 1. As each neuron in the output layer corresponds to a digit, the one neuron that ends up having the highest value will represent the result of the classification. 

In reality, we feed a network with a <i>batch</i> of inputs at a time. Here we can think of feeding the network with a batch of 100 images. So for each batch, we have a matrix of 100 rows (corresponding to one image per line, in form of a flattened vector) and 784 columns. We call this matrix X. Corresponding to this, we can define a matrix of weights W that contains the same number of rows and in each row there are 10 different weights (size 784 * 10). We can construct the matrix L by matrix multiplication of X and W and adding the corresponding bias values: $L = XW + b$. 

Bias is a constant for each neuron irregardless of the weights. Therefore for $b$ we would only have a single row of 10 different constants, and to match the shape of XW (100 by 10), we write down the same row 10 times (i.e. broadcast it on all lines) to get a 10 by 10 matrix for the biases. With this trick, we can now safely add the biases to the result of our matrix multiplication. The resulting 100 by 10 matrix, contains, on each row, the weighted sum of all inputs plus biases for each image (See <a href="https://docs.google.com/presentation/d/1TVixw6ItiZ8igjp6U17tcgoFrLSaHWQmMOwjlgQY9co/pub?slide=id.g110257a6da_0_431">this</a> for a visual representation). This value can be used as input to the softmax function: $Y = softmax(XW + b)$. The $Y$ matrix is 100 * 10, on each line it contains 10 numbers in the range (0, 1) that add up to one. These numbers could be interpreted as probabilities and the one neuron that has higher probability corresponds to the prediction of the classifier. This is how we construct this model in tensorflow:

<code>Y = tf.nn.softmax(tf.matmul(X, W) + b)</code>

Afterwards we need to asses the quality of the predictions. One way is to see the difference between the precited outputs and the intended outputs. For intended outputs, we can construct vectors that are "one-hot" encoded (<a href="https://docs.google.com/presentation/d/1TVixw6ItiZ8igjp6U17tcgoFrLSaHWQmMOwjlgQY9co/pub?slide=id.g110257a6da_0_9017">here</a>) and then compute the distance between the intended and predicted vectors. We can use several metrics for computing distance, but cross entropy distance is the standard method. Having calculated the distance, the goal will be optimizing the parameters of the model (weights) in a way that this distance would be minimized. 

In [14]:
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

# reading the data
mnist = read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

# let's define the placeholders for weights, inputs and biases

# 28 * 28 images, each pixel containing 1 value (for colored images, last value would have been 3)
# The first variable (None) will end up being the batch size (here detrmined to be 100 later)
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
# placeholder for one-hot encoded vectors (known labels)  
Y_ = tf.placeholder(tf.float32, [None, 10])
# matrix of weights (size 784 * 10)
W = tf.Variable(tf.zeros([784, 10]))
# biases, initialized to be zeros
b = tf.Variable(tf.zeros([10]))

# the model 
# X is flattened with reshape. -1 means preserve the # of elements (ends up to be the # of images in the batch)
# note the difference between Y and Y_
# Y = tf.nn.softmax(tf.matmul(tf.reshape(X, [-1, 784]), W) + b)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


We can now select an optimizer to minimize the cost function. Gradient Descent is the simplest option. We set the learning rate to be 0.003.   

In [19]:

train_step = optimizer.minimize(cross_entropy)

def train_neural_network(x):
    # run the model first
    Y = tf.nn.softmax(tf.matmul(tf.reshape(X, [-1, 784]), W) + b)
    # tensor: A typed multi-dimensional array.
    # tf.reduce_sum: computes the sum of elements across dimensions of a tensor.
    # inputs to the cross entropy loss function are Y and Y_ (computed and intended outputs)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=Y_) )
    # we'd like to minimize this cost
    optimizer = tf.train.GradientDescentOptimizer(0.003).minimize(cost)
    #optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    # how many epochs do we need (we try with 10 initilly)
    hm_epochs = 20
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss = 0 
            for _ in range(int(mnist.train.num_examples/100)):
                epoch_x, epoch_y = mnist.train.next_batch(100)
                # c: cost
                _, c = sess.run([optimizer, cost], feed_dict={X: epoch_x, Y_: epoch_y})
                epoch_loss += c
            print('epoch', epoch, 'completed out of', hm_epochs,'. loss:', epoch_loss)
            
        # Now that we have trained the model we can gauge its performance 
        # tf.argmax: returns the index with the largest value across axes of a tensor.
        # we are checking to see if that value is the same in the intended output vs prediction 
        # accuracy (cases where Y and Y_ are equal the model's prediction was right)
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print('Accuracy:', accuracy.eval({X:mnist.test.images, Y_:mnist.test.labels}))
# Now we can actually run the netwrok
train_neural_network(X)

epoch 0 completed out of 20 . loss: 1374.65232563
epoch 1 completed out of 20 . loss: 1353.44231319
epoch 2 completed out of 20 . loss: 1324.76445913
epoch 3 completed out of 20 . loss: 1291.46653032
epoch 4 completed out of 20 . loss: 1254.60885108
epoch 5 completed out of 20 . loss: 1223.1776979
epoch 6 completed out of 20 . loss: 1198.01281548
epoch 7 completed out of 20 . loss: 1177.21272779
epoch 8 completed out of 20 . loss: 1159.81693959
epoch 9 completed out of 20 . loss: 1144.68120825
epoch 10 completed out of 20 . loss: 1131.70219696
epoch 11 completed out of 20 . loss: 1120.62023306
epoch 12 completed out of 20 . loss: 1110.83436501
epoch 13 completed out of 20 . loss: 1102.23976743
epoch 14 completed out of 20 . loss: 1094.69920373
epoch 15 completed out of 20 . loss: 1088.06920874
epoch 16 completed out of 20 . loss: 1082.50450015
epoch 17 completed out of 20 . loss: 1077.44258535
epoch 18 completed out of 20 . loss: 1072.88884068
epoch 19 completed out of 20 . loss: 1069.

With an AdamOptimizer, accuracy bumps up to 93 percent, meaning choosing the proper opimizer is very important. 